In [108]:
import requests
import rich
import re

In [109]:
# Function to fetch the current stock price using Alpha Vantage
def get_stock_price(symbol, api_key):
    try:
        # Alpha Vantage API URL
        url = f"https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={symbol}&interval=1min&apikey={api_key}"
        response = requests.get(url)
        data = response.json()
        
        # Check if the response contains stock data
        if "Time Series (1min)" in data:
            # Extract the latest stock price (most recent time point)
            latest_time = list(data["Time Series (1min)"].keys())[0]
            latest_data = data["Time Series (1min)"][latest_time]
            price = latest_data["1. open"]  # Taking the 'open' price as current

            return f"The current price of {symbol.upper()} is ${price}."
        else:
            return "Invalid stock symbol or data not available."
    except requests.exceptions.RequestException as e:
        return f"Error: {str(e)}"

In [110]:
import re

# Function to process user input and extract the stock symbol
def process_user_input(user_input):
    # Match patterns like: price of AAPL, current price for TSLA, stock price MSFT, etc.
    match = re.search(r'(?:stock price|price|trading at|current price)\s*(?:of|for)?\s*(\b[A-Z]{1,5}\b)', user_input, re.IGNORECASE)
    
    if match:
        symbol = match.group(1).upper()  # Extracted symbol
        return symbol
    else:
        return None


In [111]:
# Stock price function (just added)
def invoke_stock_price(symbol):
    stock_price = get_stock_price(symbol, "TCNZL48AQVQJXADP")  # Use your Alpha Vantage API key
    return stock_price

In [112]:
# Function to fetch the current cryptocurrency price
def get_crypto_price(crypto_symbol):
    try:
        response = requests.get(f"https://api.coingecko.com/api/v3/simple/price?ids={crypto_symbol}&vs_currencies=usd")
        response.raise_for_status()
        data = response.json()
        price = data.get(crypto_symbol, {}).get('usd')
        return price if price else "Price not found"
    except requests.exceptions.RequestException as e:
        return f"API request failed: {e}"


In [113]:
import requests

def invoke_crypto_price(crypto_symbol):
    try:
        url = f"https://api.coingecko.com/api/v3/simple/price?ids={crypto_symbol}&vs_currencies=usd"
        response = requests.get(url)
        response.raise_for_status()

        data = response.json()
        price = data.get(crypto_symbol, {}).get('usd')

        if price:
            return f"The current price of {crypto_symbol.replace('-', ' ').title()} is ${price} USD."
        else:
            return f"Sorry, I couldn't find the price for {crypto_symbol}."

    except requests.exceptions.RequestException as e:
        print(f"API request failed: {e}")
        return "Sorry, there was an issue fetching the crypto price."


In [114]:
# Function to fetch the price of gold or silver
def make_gapi_request(metal="XAU"):  # XAU for gold, XAG for silver
    api_key = "goldapi-fwvgfsm6oqtc0i-io"
    curr = "USD"  # Always USD for this example
    date = ""  # If you need a specific date, you can add it here

    url = f"https://www.goldapi.io/api/{metal}/{curr}{date}"
    
    headers = {
        "x-access-token": api_key,
        "Content-Type": "application/json"
    }

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raise an error for bad responses (e.g., 400, 403, etc.)

        result = response.json()
        if 'price' in result:
            return result['price']
        else:
            return "Price not found in the response."
    except requests.exceptions.RequestException as e:
        return "Sorry, I couldn't fetch the metal price at the moment."

In [115]:
def get_metal_price(metal="gold"):
    metal_symbol = "XAU" if metal == "gold" else "XAG"  # XAU for Gold, XAG for Silver
    return make_gapi_request(metal_symbol)

def invoke_metal_price(metal="gold"):
    metal_price = get_metal_price(metal)
    if metal_price is not None:
        return f"The current price of {metal.capitalize()} is ${metal_price} USD per ounce."
    else:
        return f"Sorry, I couldn't fetch the {metal} price at the moment."

In [116]:
# Correct Mapping of Common Cryptocurrencies to CoinGecko IDs
CRYPTOCURRENCY_MAP = {
    "bitcoin": "bitcoin",
    "btc": "bitcoin",
    "ethereum": "ethereum",
    "eth": "ethereum",
    "dogecoin": "dogecoin",
    "doge": "dogecoin",
    "ripple": "ripple",
    "xrp": "ripple",
    "litecoin": "litecoin",
    "ltc": "litecoin",
    "cardano": "cardano",
    "ada": "cardano",
    "polkadot": "polkadot",
    "dot": "polkadot",
    "solana": "solana",
    "sol": "solana",
    "binance coin": "binancecoin",
    "bnb": "binancecoin",
    "shiba inu": "shiba-inu",
    "shib": "shiba-inu"
}


In [117]:
def process_tool(user_input, api_key):
    user_input_lower = user_input.lower()

    # Check for stock price queries
    if "stock price" in user_input_lower or "price" in user_input_lower or "trading at" in user_input_lower:
        symbol = process_user_input(user_input)
        if symbol:
            return get_stock_price(symbol, api_key)
        else:
            return "Sorry, I couldn't understand the stock symbol from your input. Please try again."

    # Check for gold or silver queries
    elif "gold" in user_input_lower:
        return invoke_metal_price("gold")
    elif "silver" in user_input_lower:
        return invoke_metal_price("silver")

    # Check for crypto price queries
    for crypto_name, crypto_symbol in CRYPTOCURRENCY_MAP.items():
        if crypto_name in user_input_lower:
            return invoke_crypto_price(crypto_symbol)

    return "Sorry, I couldn't understand your query. Please ask about stock price, gold, silver, or crypto."


In [118]:

# Main loop for user input handling
def main():
    api_key = "TCNZL48AQVQJXADP"  # Replace with your actual API key
    print("Welcome! You can ask about stock prices, gold, silver, or cryptocurrency prices.")
    print("To exit, type 'exit'.\n")
    
    while True:
        # Get user input
        user_input = input("Please enter your query: ").strip()
        
        # Check if the user wants to exit
        if user_input.lower() == "exit":
            print("Goodbye!")
            break
        
        # Process the user input and fetch relevant information
        response = process_tool(user_input, api_key)
        
        # Display the result
        rich.print(response)

In [119]:
# Run the main function
if __name__ == "__main__":
    main()


Welcome! You can ask about stock prices, gold, silver, or cryptocurrency prices.
To exit, type 'exit'.



The current price of TSLA is $402.0100.

The current price of GOOGL is $202.8000.

Invalid stock symbol or data not available.

The current price of Bitcoin is $95732 USD.

The current price of Ethereum is $2589.37 USD.

Goodbye!
